In [1]:
# Dependencies and Setup
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re
import time 
import datetime 
import numpy as np
from config1 import username1,password1,host1,port1,database1,census_key
import sqlalchemy
from sqlalchemy import create_engine, func, inspect, desc
pd.options.mode.chained_assignment = None  # default='warn'
import requests
import json
from pandas.io.json import json_normalize
from timeit import default_timer as timer
from census import Census
from us import states
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import statistics
import math

Using TensorFlow backend.


In [2]:
# Database Setup
start = timer()
connection1=f'{username1}:{password1}@{host1}:{port1}/{database1}'
engine1 = create_engine(f'postgresql://{connection1}')

# Pull Data
orders=pd.read_sql_query('select * from orders', con=engine1)
state_heatmap=pd.read_sql_query('select * from state_heatmap', con=engine1)
county_heatmap=pd.read_sql_query('select * from county_heatmap', con=engine1)
state_cases=pd.read_sql_query('select * from state_cases', con=engine1)
state_deaths=pd.read_sql_query('select * from state_deaths', con=engine1)
county_cases1=pd.read_sql_query('select * from county_cases1', con=engine1)
county_cases2=pd.read_sql_query('select * from county_cases2', con=engine1)
county_cases3=pd.read_sql_query('select * from county_cases3', con=engine1)
county_deaths1=pd.read_sql_query('select * from county_deaths1', con=engine1)
county_deaths2=pd.read_sql_query('select * from county_deaths2', con=engine1)
county_deaths3=pd.read_sql_query('select * from county_deaths3', con=engine1)

# Concats
county_cases=pd.concat([county_cases1,county_cases2,county_cases3], ignore_index=True)
county_deaths=pd.concat([county_deaths1,county_deaths2,county_deaths3], ignore_index=True)
end = timer()
print(end - start)

10.0711763


In [3]:
# Call Cencus Data
start = timer()
c = Census(census_key, year=2018)
census_data = c.acs5.get(('B01003_001E', 'B17001_002E','B19013_001E','B01001A_002E','B01001A_017E','B01002_001E','B02001_001E',
                          'B02001_003E','B03001_003E'), {'for': 'county:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df= census_df.rename(columns={'B01003_001E': 'Population',
                                     'B17001_002E': 'Poverty Count',
                                     'B19013_001E': 'Median Household Income',
                                     'B01001A_002E': 'male pop',
                                     'B01001A_017E':'female pop',
                                     'B01002_001E': 'median age',
                                     'B02001_001E': 'total race pop',
                                     'B02001_003E': 'black pop',
                                     'B03001_003E': 'hispanic pop',
                                     'state':'State',
                                     'county':'County'})
census_df['fips']=census_df['State']+census_df['County']
census_df['%male']=census_df['male pop']/(census_df['male pop']+census_df['female pop'])
census_df['%black']=census_df['black pop']/(census_df['total race pop'])
census_df['%hispanic']=census_df['hispanic pop']/(census_df['total race pop'])
state_census1=census_df.groupby("State")["Population",'Poverty Count','male pop','female pop','total race pop',
                                        'black pop','hispanic pop'].sum()
state_census2=census_df.groupby("State")["Median Household Income",'median age'].mean()
state_census1['%male']=state_census1['male pop']/(state_census1['male pop']+state_census1['female pop'])
state_census1['%black']=state_census1['black pop']/(state_census1['total race pop'])
state_census1['%hispanic']=state_census1['hispanic pop']/(state_census1['total race pop'])
census_df=census_df.drop(['male pop','female pop','total race pop','black pop','hispanic pop'], axis=1)
state_census1=state_census1.drop(['male pop','female pop','total race pop','black pop','hispanic pop'], axis=1)
state_census1=state_census1.reset_index()
state_census2=state_census2.reset_index()
state_census1=state_census1.rename(columns={'State':'fips'})
state_census2=state_census2.rename(columns={'State':'fips'})
state_census=state_census1.merge(state_census2,on='fips')
end = timer()
print(end - start)

5.5179384


C:\Users\Henry Randall\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\Henry Randall\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [4]:
start = timer()
state_area={}
with open('stateborders.json') as json_file:
    data = json.load(json_file)
data=data['features']
k=0
for state in data:
    state_area[k] = {"state":data[k]['properties']['NAME'],"fips": data[k]['properties']['STATE'],
                     "area": data[k]['properties']['CENSUSAREA']}
    k=k+1
state_area = pd.DataFrame.from_dict(state_area, "index")

county_area={}
with open('countyborders.json') as json_file:
    data = json.load(json_file)
data=data['features']
k=0
for county in data:
    county_area[k] = {"fips": (data[k]['properties']['STATE'])+(data[k]['properties']['COUNTY']),
                      "area": data[k]['properties']['CENSUSAREA']}
    k=k+1
county_area = pd.DataFrame.from_dict(county_area, "index")
end = timer()
print(end - start)

1.0260071999999987


In [5]:
start = timer()
state_census=state_census.merge(state_area, on='fips')
state_census.iloc[8,8]='District Of Columbia'
county_census=census_df.merge(county_area, on='fips')
county_census.iloc[291,1]=10300
county_census.iloc[291,2]=33422
state_census['Poverty Rate'] = 100 * \
    state_census['Poverty Count'].astype(
        int) / state_census['Population'].astype(int)
state_census['Population Density'] = 1 * \
    state_census['Population'].astype(
        int) / state_census['area'].astype(int)
state_census=state_census.drop(['Poverty Count','area'], axis=1)
# Convert Poverty Copunt to Poverty Rate (Poverty Count / Population)
county_census['Poverty Rate'] = 100 * \
    county_census['Poverty Count'].astype(
        int) / county_census['Population'].astype(int)
county_census['Population Density'] = 1 * \
    county_census['Population'].astype(
        int) / county_census['area'].astype(int)
county_census=county_census.drop(['Poverty Count','area','State','County'], axis=1)
end = timer()
print(end - start)

0.01188020000000023


In [6]:
county_heatmap=county_heatmap.merge(county_census,on='fips')
state_heatmap=state_heatmap.merge(state_census,on='state')

In [7]:
latdata_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
latdata=pd.read_csv(latdata_url, error_bad_lines=False)
latdata=latdata.rename(columns={'Province_State': 'State'})
latdata=latdata.reset_index()
latdata=latdata.drop(columns=['index'])
latdata.dropna(subset = ["Admin2"], inplace=True)
latdata=latdata[latdata.Admin2 != 'Unassigned']
latdata=latdata.dropna()
latdata=latdata[~latdata['Admin2'].astype(str).str.startswith('Out of')]
latdata=latdata[latdata.Admin2 != 'Out of*']
latdata=latdata.reset_index()
latdata['FIPS']=latdata.FIPS.map('{0:0>5.0f}'.format)
latdata=latdata[['State','FIPS','Lat','Long_']]
state_latdata=latdata.groupby('State').mean()
latdata=latdata.drop(columns=['State'])
state_latdata=state_latdata.reset_index()

In [8]:
# Load Models
cases_model = keras.models.load_model("Cases")
deaths_model = keras.models.load_model("Deaths")

Start Loops Here

In [9]:
start = timer()
# Set number of days to predict
n=14
for q in range(n):
    # Trim Matrix to only what we need for the models
    county_cases_cut=county_cases.copy()
    county_deaths_cut=county_deaths.copy()
    state_cases_cut=state_cases.copy()
    state_deaths_cut=state_deaths.copy()
    county_cases_cut.drop(county_cases_cut.columns[3:-14],axis=1,inplace=True)
    county_deaths_cut.drop(county_deaths_cut.columns[3:-8],axis=1,inplace=True)
    state_cases_cut.drop(state_cases_cut.columns[1:-14],axis=1,inplace=True)
    state_deaths_cut.drop(state_deaths_cut.columns[1:-8],axis=1,inplace=True)

    #Find daily totals - county cases
    county_cases_cut_daily=county_cases_cut.copy()
    [r,c]=county_cases_cut_daily.shape
    for j in range (0,r):
        last=0
        for i in range (3,c):
            current=county_cases_cut_daily.iloc[j,i]-last
            last=county_cases_cut_daily.iloc[j,i]
            county_cases_cut_daily.iat[j,i]=current
    county_cases_cut_daily.drop(county_cases_cut_daily.columns[3],axis=1,inplace=True)     

    #Find daily totals - county deaths
    county_deaths_cut_daily=county_deaths_cut.copy()
    [r,c]=county_deaths_cut_daily.shape
    for j in range (0,r):
        last=0
        for i in range (3,c):
            current=county_deaths_cut_daily.iloc[j,i]-last
            last=county_deaths_cut_daily.iloc[j,i]
            county_deaths_cut_daily.iat[j,i]=current    
    county_deaths_cut_daily.drop(county_deaths_cut_daily.columns[3],axis=1,inplace=True)

    #Find daily totals - state cases
    state_cases_cut_daily=state_cases_cut.copy()
    [r,c]=state_cases_cut_daily.shape
    for j in range (0,r):
        last=0
        for i in range (1,c):
            current=state_cases_cut_daily.iloc[j,i]-last
            last=state_cases_cut_daily.iloc[j,i]
            state_cases_cut_daily.iat[j,i]=current
    state_cases_cut_daily.drop(state_cases_cut_daily.columns[1],axis=1,inplace=True)

    #Find daily totals - state deaths
    state_deaths_cut_daily=state_deaths_cut.copy()
    [r,c]=state_deaths_cut_daily.shape
    for j in range (0,r):
        last=0
        for i in range (1,c):
            current=state_deaths_cut_daily.iloc[j,i]-last
            last=state_deaths_cut_daily.iloc[j,i]
            state_deaths_cut_daily.iat[j,i]=current
    state_deaths_cut_daily.drop(state_deaths_cut_daily.columns[1],axis=1,inplace=True)

    #Find 7day totals - county cases
    county_cases_cut_7day=county_cases_cut_daily.copy()
    [r,c]=county_cases_cut_7day.shape
    for j in range (0,r):
        for i in range (9,c):
            county_cases_cut_7day.iloc[j,i]=statistics.mean([county_cases_cut_daily.iloc[j,i-6],county_cases_cut_daily.iloc[j,i-5],
                    county_cases_cut_daily.iloc[j,i-4],county_cases_cut_daily.iloc[j,i-3],county_cases_cut_daily.iloc[j,i-3],
                    county_cases_cut_daily.iloc[j,i-1],county_cases_cut_daily.iloc[j,i]])
    county_cases_cut_7day = county_cases_cut_7day.drop(county_cases_cut_7day.columns[3:9],axis=1)

    #Find 7day totals - county deaths
    county_deaths_cut_7day=county_deaths_cut_daily.copy()
    [r,c]=county_deaths_cut_7day.shape
    for j in range (0,r):
        last=0
        for i in range (9,c):
            county_deaths_cut_7day.iloc[j,i]=statistics.mean([county_deaths_cut_daily.iloc[j,i-6],county_deaths_cut_daily.iloc[j,i-5],
                    county_deaths_cut_daily.iloc[j,i-4],county_deaths_cut_daily.iloc[j,i-3],county_deaths_cut_daily.iloc[j,i-3],
                    county_deaths_cut_daily.iloc[j,i-1],county_deaths_cut_daily.iloc[j,i]])
    county_deaths_cut_7day = county_deaths_cut_7day.drop(county_deaths_cut_7day.columns[3:9],axis=1)

    #Find 7day totals - state cases
    state_cases_cut_7day=state_cases_cut_daily.copy()
    [r,c]=state_cases_cut_7day.shape
    for j in range (0,r):
        for i in range (7,c):
            state_cases_cut_7day.iloc[j,i]=statistics.mean([state_cases_cut_daily.iloc[j,i-6],state_cases_cut_daily.iloc[j,i-5],
                    state_cases_cut_daily.iloc[j,i-4],state_cases_cut_daily.iloc[j,i-3],state_cases_cut_daily.iloc[j,i-2],
                    state_cases_cut_daily.iloc[j,i-1],state_cases_cut_daily.iloc[j,i]])
    state_cases_cut_7day = state_cases_cut_7day.drop(state_cases_cut_7day.columns[1:7],axis=1)

    #Find 7day totals - county deaths
    state_deaths_cut_7day=state_deaths_cut_daily.copy()
    [r,c]=state_deaths_cut_7day.shape
    for j in range (0,r):
        last=0
        for i in range (7,c):
            state_deaths_cut_7day.iloc[j,i]=statistics.mean([state_deaths_cut_daily.iloc[j,i-6],state_deaths_cut_daily.iloc[j,i-5],
                    state_deaths_cut_daily.iloc[j,i-4],state_deaths_cut_daily.iloc[j,i-3],state_deaths_cut_daily.iloc[j,i-2],
                    state_deaths_cut_daily.iloc[j,i-1],state_deaths_cut_daily.iloc[j,i]])
    state_deaths_cut_7day = state_deaths_cut_7day.drop(state_deaths_cut_7day.columns[1:7],axis=1)

    # Cut out day 1 cases for the death model
    countycases1_cut_7day=county_cases_cut_7day.copy()
    statecases1_cut_7day=state_cases_cut_7day.copy()
    countycases1_cut_7day.drop(countycases1_cut_7day.columns[4:],axis=1,inplace=True)
    statecases1_cut_7day.drop(statecases1_cut_7day.columns[2:],axis=1,inplace=True)
    county_cases_cut_7day.drop(county_cases_cut_7day.columns[3:-1],axis=1,inplace=True)
    state_cases_cut_7day.drop(state_cases_cut_7day.columns[1:-1],axis=1,inplace=True)

    # Proximity Cases - State
    rad5_state_7day={}
    [r,c]=state_cases_cut_7day.shape
    for j in range (0,r):
        lat=state_latdata.iloc[j,1]
        lon=state_latdata.iloc[j,2]
        index5=[]
        for k in range (0,r):
            dist=math.sqrt(((state_latdata.iloc[k,1]-lat)**2)+((state_latdata.iloc[k,2]-lon)**2))
            if dist<5:
                index5.append(state_latdata.iloc[k,0])
        rad5_state_7day[j]=((((state_cases_cut_7day[state_cases_cut_7day['State'].isin(index5)]).sum())[1:])/(((state_heatmap[state_heatmap['state'].str.lower().isin([x.lower() for x in index5])]).sum())['population']))*100000
    rad5_state_7day = pd.DataFrame.from_dict(rad5_state_7day, "index")
    rad5_state_7day.insert(0, 'State', state_cases_cut_7day['State'])
    rad5_state_7day = rad5_state_7day.fillna(0)

    # Proximity Cases - County
    rad5_county_7day={}
    [r,c]=county_cases_cut_7day.shape
    for j in range (0,r):
        lat=latdata.iloc[j,1]
        lon=latdata.iloc[j,2]
        index5=[]
        for k in range (0,r):
            dist=math.sqrt(((latdata.iloc[k,1]-lat)**2)+((latdata.iloc[k,2]-lon)**2))
            if dist<5:
                index5.append(latdata.iloc[k,0])
        rad5_county_7day[j]=((((county_cases_cut_7day[county_cases_cut_7day['FIPS'].isin(index5)]).sum())[3:])/(((county_heatmap[county_heatmap['fips'].isin(index5)]).sum())['population']))*100000
    rad5_county_7day = pd.DataFrame.from_dict(rad5_county_7day, "index")
    rad5_county_7day.insert(0, 'FIPS', county_cases_cut_7day['FIPS'])
    rad5_county_7day.insert(1, 'County', county_cases_cut_7day['County'])
    rad5_county_7day.insert(2, 'State', county_cases_cut_7day['State'])
    rad5_county_7day = rad5_county_7day.fillna(0)

    # Trim Dataframes
    county_cases_cut.drop(county_cases_cut.columns[1:-1],axis=1,inplace=True)
    county_deaths_cut.drop(county_deaths_cut.columns[1:-1],axis=1,inplace=True)
    state_cases_cut.drop(state_cases_cut.columns[1:-1],axis=1,inplace=True)
    state_deaths_cut.drop(state_deaths_cut.columns[1:-1],axis=1,inplace=True)

    #  State Cases Forecast
    state_predict_cases=state_heatmap[['state','Population','Population Density','%black','%hispanic','median age']]
    state_predict_deaths=state_heatmap[['state','Population','Population Density','%black','%hispanic','Median Household Income',
                                        'median age']]

    state_predict_cases.iloc[8,0]='District of Columbia'
    state_predict_deaths.iloc[8,0]='District of Columbia'

    state_predict_cases=state_predict_cases.merge(state_cases_cut, left_on='state', right_on='State')
    state_predict_cases.drop('State',axis=1,inplace=True)
    state_predict_cases.columns = [*state_predict_cases.columns[:-1], 'day7']
    state_predict_cases['day7']=(state_predict_cases['day7']*10000)/state_predict_cases['Population']

    state_predict_cases=state_predict_cases.merge(rad5_state_7day, left_on='state', right_on='State')
    state_predict_cases.drop('State',axis=1,inplace=True)
    state_predict_cases.columns = [*state_predict_cases.columns[:-1], 'rad5']

    state_predict_cases=state_predict_cases.merge(state_cases_cut_7day, left_on='state', right_on='State')
    state_predict_cases.columns = [*state_predict_cases.columns[:-1], '7day7']
    state_predict_cases['7day7']=(state_predict_cases['7day7']*10000)/state_predict_cases['Population']
    state_predict_cases.drop('State',axis=1,inplace=True)

    # State Death Forecast
    state_predict_deaths=state_predict_deaths.merge(statecases1_cut_7day, left_on='state', right_on='State')
    state_predict_deaths.columns = [*state_predict_deaths.columns[:-1], 'casesday1']
    state_predict_deaths.drop('State',axis=1,inplace=True)
    state_predict_deaths['casesday1']=(state_predict_deaths['casesday1']*10000)/state_predict_deaths['Population']

    state_predict_deaths=state_predict_deaths.merge(state_deaths_cut, left_on='state', right_on='State')
    state_predict_deaths.columns = [*state_predict_deaths.columns[:-1], 'day7']
    state_predict_deaths.drop('State',axis=1,inplace=True)
    state_predict_deaths['day7']=(state_predict_deaths['day7']*10000)/state_predict_deaths['Population']

    state_predict_deaths=state_predict_deaths.merge(state_deaths_cut_7day, left_on='state', right_on='State')
    state_predict_deaths.columns = [*state_predict_deaths.columns[:-1], '7day7']
    state_predict_deaths['7day7']=(state_predict_deaths['7day7']*10000)/state_predict_deaths['Population']
    state_predict_deaths.drop('State',axis=1,inplace=True)

    # County Cases Forecast
    county_predict_cases=county_heatmap[['fips','Population','Population Density','%black','%hispanic','median age']]
    county_predict_deaths=county_heatmap[['fips','Population','Population Density','%black','%hispanic','Median Household Income',
                                        'median age']]

    county_predict_cases=county_predict_cases.merge(county_cases_cut, left_on='fips', right_on='FIPS')
    county_predict_cases.drop('FIPS',axis=1,inplace=True)
    county_predict_cases.columns = [*county_predict_cases.columns[:-1], 'day7']
    county_predict_cases['day7']=(county_predict_cases['day7']*10000)/county_predict_cases['Population']

    county_predict_cases=county_predict_cases.merge(rad5_county_7day, left_on='fips', right_on='FIPS')
    county_predict_cases.drop('FIPS',axis=1,inplace=True)
    county_predict_cases.columns = [*county_predict_cases.columns[:-1], 'rad5']
    county_predict_cases.drop('State',axis=1,inplace=True)
    county_predict_cases.drop('County',axis=1,inplace=True)

    county_predict_cases=county_predict_cases.merge(county_cases_cut_7day, left_on='fips', right_on='FIPS')
    county_predict_cases.drop('FIPS',axis=1,inplace=True)
    county_predict_cases.columns = [*county_predict_cases.columns[:-1], '7day7']
    county_predict_cases['7day7']=(county_predict_cases['7day7']*10000)/county_predict_cases['Population']
    county_predict_cases.drop('State',axis=1,inplace=True)
    county_predict_cases.drop('County',axis=1,inplace=True)

    # County Deaths forecast
    county_predict_deaths=county_predict_deaths.merge(countycases1_cut_7day, left_on='fips', right_on='FIPS')
    county_predict_deaths.columns = [*county_predict_deaths.columns[:-1], 'casesday1']
    county_predict_deaths.drop('FIPS',axis=1,inplace=True)
    county_predict_deaths['casesday1']=(county_predict_deaths['casesday1']*10000)/county_predict_deaths['Population']
    county_predict_deaths.drop('State',axis=1,inplace=True)
    county_predict_deaths.drop('County',axis=1,inplace=True)

    county_predict_deaths=county_predict_deaths.merge(county_deaths_cut, left_on='fips', right_on='FIPS')
    county_predict_deaths.columns = [*county_predict_deaths.columns[:-1], 'day7']
    county_predict_deaths.drop('FIPS',axis=1,inplace=True)
    county_predict_deaths['day7']=(county_predict_deaths['day7']*10000)/county_predict_deaths['Population']

    county_predict_deaths=county_predict_deaths.merge(county_deaths_cut_7day, left_on='fips', right_on='FIPS')
    county_predict_deaths.columns = [*county_predict_deaths.columns[:-1], '7day7']
    county_predict_deaths['7day7']=(county_predict_deaths['7day7']*10000)/county_predict_deaths['Population']
    county_predict_deaths.drop('FIPS',axis=1,inplace=True)
    county_predict_deaths.drop('State',axis=1,inplace=True)
    county_predict_deaths.drop('County',axis=1,inplace=True)

    # Predict State Deaths
    state_predict_deaths1=state_predict_deaths.copy()
    state_predict_deaths1.drop(state_predict_deaths1.columns[:2],axis=1,inplace=True)
    state_predict_deaths['prediction']= deaths_model.predict(state_predict_deaths1)
    state_predict_deaths['prediction']=((state_predict_deaths['prediction']*state_predict_deaths['Population']))/10000
    state_predict_deaths.drop(state_predict_deaths.columns[1:-1],axis=1,inplace=True)
    state_predict_deaths['prediction']=state_predict_deaths['prediction'].astype(np.int64)
    num = state_predict_deaths._get_numeric_data()
    num[num < 0] = 0

    # Predict State Cases
    state_predict_cases1=state_predict_cases.copy()
    state_predict_cases1.drop(state_predict_cases1.columns[:2],axis=1,inplace=True)
    state_predict_cases['prediction']= cases_model.predict(state_predict_cases1)
    state_predict_cases['prediction']=((state_predict_cases['prediction']*state_predict_cases['Population']))/10000
    state_predict_cases.drop(state_predict_cases.columns[1:-1],axis=1,inplace=True)
    state_predict_cases['prediction']=state_predict_cases['prediction'].astype(np.int64)
    num = state_predict_cases._get_numeric_data()
    num[num < 0] = 0

    # Predict County Deaths
    county_predict_deaths1=county_predict_deaths.copy()
    county_predict_deaths1.drop(county_predict_deaths1.columns[:2],axis=1,inplace=True)
    county_predict_deaths['prediction']= deaths_model.predict(county_predict_deaths1)
    county_predict_deaths['prediction']=((county_predict_deaths['prediction']*county_predict_deaths['Population']))/10000
    county_predict_deaths.drop(county_predict_deaths.columns[1:-1],axis=1,inplace=True)
    county_predict_deaths['prediction']=county_predict_deaths['prediction'].astype(np.int64)
    num = county_predict_deaths._get_numeric_data()
    num[num < 0] = 0

    # Predict County Cases
    county_predict_cases1=county_predict_cases.copy()
    county_predict_cases1.drop(county_predict_cases1.columns[:2],axis=1,inplace=True)
    county_predict_cases['prediction']= cases_model.predict(county_predict_cases1)
    county_predict_cases['prediction']=((county_predict_cases['prediction']*county_predict_cases['Population']))/10000
    county_predict_cases.drop(county_predict_cases.columns[1:-1],axis=1,inplace=True)
    county_predict_cases['prediction']=county_predict_cases['prediction'].astype(np.int64)
    num = county_predict_cases._get_numeric_data()
    num[num < 0] = 0

    # Merge State Cases
    state_cases=state_cases.merge(state_predict_cases,left_on='State',right_on='state')
    state_cases.drop('state',axis=1,inplace=True)
    state_cases['prediction']=state_cases['prediction']+state_cases.iloc[:,-2]
    newdate=pd.to_datetime(state_cases.columns[-2])
    newdate=newdate+pd.DateOffset(1)
    newdate= newdate.strftime("%#m/%d/%y")
    state_cases = state_cases.rename(columns={"prediction": newdate})

    # Merge State Deaths
    state_deaths=state_deaths.merge(state_predict_deaths,left_on='State',right_on='state')
    state_deaths.drop('state',axis=1,inplace=True)
    state_deaths['prediction']=state_deaths['prediction']+state_deaths.iloc[:,-2]
    newdate=pd.to_datetime(state_deaths.columns[-2])
    newdate=newdate+pd.DateOffset(1)
    newdate= newdate.strftime("%#m/%d/%y")
    state_deaths = state_deaths.rename(columns={"prediction": newdate})

    # Merge County Cases
    county_cases=county_cases.merge(county_predict_cases,left_on='FIPS',right_on='fips')
    county_cases.drop('fips',axis=1,inplace=True)
    county_cases['prediction']=county_cases['prediction']+county_cases.iloc[:,-2]
    newdate=pd.to_datetime(county_cases.columns[-2])
    newdate=newdate+pd.DateOffset(1)
    newdate= newdate.strftime("%#m/%d/%y")
    county_cases=county_cases.rename(columns={"prediction": newdate})

    # Merge County Death
    county_deaths=county_deaths.merge(county_predict_deaths,left_on='FIPS',right_on='fips')
    county_deaths.drop('fips',axis=1,inplace=True)
    county_deaths['prediction']=county_deaths['prediction']+county_deaths.iloc[:,-2]
    newdate=pd.to_datetime(county_deaths.columns[-2])
    newdate=newdate+pd.DateOffset(1)
    newdate= newdate.strftime("%#m/%d/%y")
    county_deaths = county_deaths.rename(columns={"prediction": newdate})

end = timer()
print(end - start)

2097.2675296


In [10]:
# Split Arrays
[county_cases1,county_cases2,county_cases3]=np.array_split(county_cases, 3)
[county_deaths1,county_deaths2,county_deaths3]=np.array_split(county_deaths,3)

In [11]:
county_cases1.to_sql(name='county_cases1', con=engine1, if_exists='replace', index=False)
county_cases2.to_sql(name='county_cases2', con=engine1, if_exists='replace', index=False)
county_cases3.to_sql(name='county_cases3', con=engine1, if_exists='replace', index=False)

In [12]:
county_deaths1.to_sql(name='county_deaths1', con=engine1, if_exists='replace', index=False)
county_deaths2.to_sql(name='county_deaths2', con=engine1, if_exists='replace', index=False)
county_deaths3.to_sql(name='county_deaths3', con=engine1, if_exists='replace', index=False)

In [13]:
state_cases.to_sql(name='state_cases', con=engine1, if_exists='replace', index=False)
state_deaths.to_sql(name='state_deaths', con=engine1, if_exists='replace', index=False)